### Задачи к Лекции 4

__Исходные данные__ 

Дан файл **"mlbootcamp5_train.csv"**. В нем содержатся данные об опросе 70000 пациентов с целью определения наличия заболеваний сердечно-сосудистой системы (ССЗ). Данные в файле промаркированы и если у человека имееются ССЗ, то значение **cardio** будет равно 1, в противном случае - 0. Описание и значения полей представлены во второй лекции.

__Загрузка файла__

In [77]:
%matplotlib inline
import numpy as np
import pandas as pd
import seaborn as sns
import scipy as sp
import math
import sklearn
from matplotlib import pyplot as plt
import warnings
from sklearn.model_selection import train_test_split
warnings.filterwarnings('ignore')
import matplotlib.pyplot as plt
plt.rcParams['figure.figsize'] = [10, 5]


df = pd.read_csv("../data/mlbootcamp5_train.csv", 
                 sep=";", 
                 index_col="id")
df = df.dropna(subset=['age','gender','height','weight','ap_hi','ap_lo','cholesterol','gluc','smoke','alco','active']).sort_values('id')
df_cat = df[(df["ap_hi"] >= 100) &
          (df["ap_hi"] <= 200) &
          (df["ap_lo"] >= 50) & 
          (df["ap_lo"] <= 150) &
          (df["weight"] >= 40)
         ]
# Делаем пол бинарным признаком
df_cat["gender_bin"] = df_cat["gender"].map({1: 0, 2: 1})
df_cat = df_cat.drop(["gender"],axis=1)
target_cat = df_cat["cardio"]


# Делаем one-hot кодирование
chol = pd.get_dummies(df_cat["cholesterol"], prefix="chol")
gluc = pd.get_dummies(df_cat["gluc"], prefix="gluc")
df_one_hot = pd.concat([df_cat, chol, gluc], axis=1)

target_data = df_one_hot["cardio"]
df_one_hot.head()
#df_cat.head()

,age,height,weight,ap_hi,ap_lo,cholesterol,gluc,smoke,alco,active,cardio,gender_bin,chol_1,chol_2,chol_3,gluc_1,gluc_2,gluc_3
id,,,,,,,,,,,,,,,,,,
0,18393,168,62.0,110,80,1,1,0,0,1,0,1,1,0,0,1,0,0
1,20228,156,85.0,140,90,3,1,0,0,1,1,0,0,0,1,1,0,0
2,18857,165,64.0,130,70,3,1,0,0,0,1,0,0,0,1,1,0,0
3,17623,169,82.0,150,100,1,1,0,0,1,1,1,1,0,0,1,0,0
4,17474,156,56.0,100,60,1,1,0,0,0,0,0,1,0,0,1,0,0


## Задачи

__1. Хоть в sklearn и присутствует реализация метода k-ближайших соседей, я же предлагаю попробовать вам написать его самостоятельно.__

* __создать классификатор используя только pandas, numpy и scipy. Гиперпараметром данного классификатора должно быть число ближайших соседей. (Необязательно) можно добавить метрику расстояния и выбор весов.__
* __С помощью кросс-валидации найти оптимальное количество ближайших соседей и (необязательно) набор признаков.__

Алгоритм работы классификатора:
 1. Для заданного прецедент  $\vec{x}$ мы считаем расстояние до всех прецедентов в обучающей выборке.
 2. Сортируем прецеденты по расстоянию до $\vec{x}$.
 3. Отбираем $k$ минимальных значений
 4. Устраиваем голосование между отобранными прецедент.

**Комментарии:** Ваши комментарии здесь.

**2. Определить какой из трех классификаторов (kNN, наивный Байес, решающее дерево) лучший в каждой метрике по отдельности: accuracy, F1-мера, ROC AUC, функция потерь. Использовать набор признаков: 'age', 'weight', 'height', 'ap_lo', 'ap_hi'.**

**(Необязательно) Найти оптимальный набор признаков.**

In [ ]:
%time
from sklearn.model_selection import train_test_split
from sklearn.model_selection import cross_validate, cross_val_score
from sklearn.base import BaseEstimator, ClassifierMixin
from sklearn.utils.multiclass import unique_labels


def df_Min_Stand(df):
    df_MinMax = df.apply(lambda x: (x-min(x))/(max(x)-min(x)))
    df_Stand = df.apply(lambda x: (x-np.mean(x))/np.std(x))
    df_list = [df_MinMax,df_Stand]
    return df_list

#Euclidean distance
def dist (a, b):
    ab_sum = 0
    for i in range(a.shape[0]):
        if (a[i] != 'cardio' and b[i] != 'cardio'):
            #print(str(a[i]) + '------' + str(b[i]) +'--i= '+str(i))
            ab_sum += (a[i] - b[i])**2
            distance = np.sqrt(ab_sum)
    return distance    

# Классификатор по методу k - наименьших соседей
class KNN(BaseEstimator, ClassifierMixin):
    
    def __init__(self, k=3):
        self.k = k
    
    def fit(self, X, y):
        self.train = X
        self.target_train = y
        self.classes_ = unique_labels(y)
        #return self

    def predict(self, X_test):
        pred_labels = [self._predict(x) for x in X_test.to_numpy()]
        return np.array(pred_labels)
    
    def _predict(self, x_test):
        dists=[dist(row_train, x_test) for index_train,row_train in self.train.iterrows()]
        k_inc = np.argsort(dists)[:self.k]
        print(k_inc)
        k_nrst_labels = [self.target_train.iloc[i] for i in k_inc]
        most_comm = Counter(k_nrst_labels).most_common(1)
        print (most_comm[0][0])
        return most_comm[0][0]
    
dfmm = df_Min_Stand(df_cat)[0]
target = dfmm['cardio']
train, test, target_train, target_test = train_test_split(dfmm, target, test_size = 0.05) 

clf_knn = KNN(k = 3)
clf_knn.fit(train,target_train)
preds = clf_knn.predict(test)

#check_estimator(KNN())
#cv = cross_val_score(KNN(), 
#                    train.loc[:, dfmm.columns != 'cardio'], 
#                    target_train,
#                    n_jobs=8,
#                    scoring='accuracy',
#                    cv=5)
#print(cv, cv.mean())

#preds
print('IT\'S TIME TO STOP')

Wall time: 0 ns
[55900 30847 63142]
1.0


In [3]:

from sklearn.naive_bayes import GaussianNB
from sklearn.tree import DecisionTreeClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import cross_validate
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import log_loss


data = df[['age', 'weight', 'height', 'ap_lo', 'ap_hi']]
target_comp = df['cardio']


classifiers = [("Tree (depth=%d)" % i, DecisionTreeClassifier(max_depth=i, random_state=13)) for i in range(1, 15)]
classifiers.append(("GaussianNB()", GaussianNB()))
for i in range(1, 15):
    classifiers.append(("kNN (neighbors=%d)" % i, KNeighborsClassifier(n_neighbors=i))) 

results = []
ll_list = []
for name, classifier in classifiers:
    cv = cross_validate(classifier, data, target_comp, n_jobs=-1, cv=5, scoring=['accuracy', 'f1', 'roc_auc'])
    results.append((name, cv["test_f1"].mean(), cv["test_accuracy"].mean(), cv["test_roc_auc"].mean()))
    model = classifier.fit(data, target_comp)
    predict = model.predict_proba(data)[:,1]
    ll_list.append(log_loss(target_comp, predict, eps = 1e-15, normalize = True))


dr = pd.DataFrame(results, columns=["Name", "F1", "Accuracy", "ROC AUC"])
dr['Log_Loss'] = pd.Series(ll_list, index=dr.index)
print("ROC_AUC\n", dr.sort_values(by="ROC AUC")[-1:])
print()
print("Accuracy\n", dr.sort_values(by="Accuracy")[-1:])
print()
print("F1\n", dr.sort_values(by="F1")[-1:])
print()
print("LogLoss\n", dr.sort_values(by="Log_Loss")[-1:])
print()
dr


'\nfrom sklearn.naive_bayes import GaussianNB\nfrom sklearn.tree import DecisionTreeClassifier\nfrom sklearn.neighbors import KNeighborsClassifier\nfrom sklearn.model_selection import cross_validate\nfrom sklearn.model_selection import GridSearchCV\nfrom sklearn.metrics import log_loss\n\n\ndata = df[[\'age\', \'weight\', \'height\', \'ap_lo\', \'ap_hi\']]\ntarget_comp = df[\'cardio\']\n\n\nclassifiers = [("Tree (depth=%d)" % i, DecisionTreeClassifier(max_depth=i, random_state=13)) for i in range(1, 15)]\nclassifiers.append(("GaussianNB()", GaussianNB()))\nfor i in range(1, 15):\n    classifiers.append(("kNN (neighbors=%d)" % i, KNeighborsClassifier(n_neighbors=i))) \n\nresults = []\nll_list = []\nfor name, classifier in classifiers:\n    cv = cross_validate(classifier, data, target_comp, n_jobs=-1, cv=5, scoring=[\'accuracy\', \'f1\', \'roc_auc\'])\n    results.append((name, cv["test_f1"].mean(), cv["test_accuracy"].mean(), cv["test_roc_auc"].mean()))\n    model = classifier.fit(data,

**Комментарии:** Ваши комментарии здесь.